---
author: Nazmus Salehin
date: 12-01-2024
title: "Human CellRank"
---

In [1]:
import cellrank as cr
import scanpy as sc

sc.settings.set_figure_params(frameon=False, dpi=100)
cr.settings.verbosity = 2

import warnings
warnings.simplefilter("ignore", category=UserWarning)

import scvelo as scv
scv.set_figure_params('scvelo')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
human_adata = sc.read('../results/02_human_integration/scvi/adata.h5ad')

In [3]:
human_adata.obs.head()

,day,ct,experiment,technology,n_counts,n_genes,ct_fine,batch,stage,timepoint,ct_orig,_scvi_batch,_scvi_labels
index,,,,,,,,,,,,,
ERX3015937_ERX3015937,5.0,Unknown,"Meistermann et al., 2021",SMARTSeq2,708313.0,5761,Unknown,"Meistermann et al., 2021",Unknown,E5.0,Unknown,0,0
ERX3015939_ERX3015939,5.0,Unknown,"Meistermann et al., 2021",SMARTSeq2,402557.0,5689,Unknown,"Meistermann et al., 2021",Unknown,E5.0,Unknown,0,0
ERX3015940_ERX3015940,5.0,Unknown,"Meistermann et al., 2021",SMARTSeq2,511338.0,6039,Unknown,"Meistermann et al., 2021",Unknown,E5.0,Unknown,0,0
ERX3015941_ERX3015941,5.0,Unknown,"Meistermann et al., 2021",SMARTSeq2,994383.0,8383,Unknown,"Meistermann et al., 2021",Unknown,E5.0,Unknown,0,0
ERX3015936_ERX3015936,5.0,Unknown,"Meistermann et al., 2021",SMARTSeq2,1389486.0,7762,Unknown,"Meistermann et al., 2021",Unknown,E5.0,Unknown,0,0


In [4]:
sc.pp.neighbors(human_adata, use_rep='X_scVI')
sc.tl.diffmap(human_adata)

sc.tl.paga(human_adata, groups='C_scANVI_nsamples')
sc.pl.paga(human_adata, color=['C_scANVI_nsamples'], frameon=False, fontoutline=True)
sc.tl.draw_graph(human_adata, init_pos='paga', n_jobs=10)

2024-01-12 13:25:32.739451: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


KeyError: "`groups` key 'C_scANVI_nsamples' not found in `adata.obs`."

In [ ]:
sc.pl.paga(human_adata, color=['C_scANVI_nsamples'], frameon=False, fontoutline=True, threshold=0.2)

In [ ]:
sc.pl.embedding(human_adata, basis="X_draw_graph_fa", color=["day"])

## Cellrank experimental time kernel

In [ ]:
human_adata.obs.day.value_counts()

In [ ]:
human_adata_8C_and_later = human_adata[human_adata.obs.day >= 3].copy()

In [ ]:
from cellrank.kernels import RealTimeKernel

In [ ]:
human_adata.obs['day_categorical'] = human_adata.obs['day'].astype('category')

In [ ]:
from moscot.problems.time import TemporalProblem

In [ ]:
tp = TemporalProblem(human_adata_8C_and_later)

In [ ]:
tp = tp.prepare(time_key="day_categorical")

In [ ]:
tp = tp.solve(epsilon=1e-3, tau_a=0.95, scale_cost="mean")

In [ ]:
tmk = RealTimeKernel.from_moscot(tp)

In [ ]:
tmk.compute_transition_matrix(self_transitions="all", conn_weight=0.2, threshold="auto")

In [ ]:
tmk.plot_random_walks(
    max_iter=500,
    start_ixs={"day_categorical": 3.0},
    basis="draw_graph_fa",
    seed=0,
    dpi=150,
    size=30,
)

In [ ]:
from cellrank.kernels import PseudotimeKernel
pk = PseudotimeKernel(human_adata, time_key="day")
pk.compute_transition_matrix()

In [ ]:
pk.plot_random_walks(
    seed=0,
    n_sims=100,
    start_ixs={"C_scANVI_nsamples": "Prelineage"},
    basis="X_draw_graph_fa",
    legend_loc="right",
    dpi=150,
)

In [ ]:
pk.plot_projection(basis="X_draw_graph_fa", recompute=True, color='C_scANVI_nsamples')